# ダブルクロスバリデーションの確認

scikit-learn のライブラリーを使用した、ダブルクロスバリデーションの手法についての確認です。

こちらの記述（というよりプロダクトの宣伝？）を参考にさせていただきました。

http://univprof.com/archives/16-06-12-3889388.html

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/26 AM 11:49:26 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/03/26 AM 11:49:26 TextArray#__init__ start
2017/03/26 AM 11:49:35 TextArray#to_vec start
2017/03/26 AM 11:49:35 TextArray#to_vec end


In [5]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=17443, feature=1967


## (3) ダブルクロスバリデーションを使用しない場合

### (3-1) モデル選択フェーズ

C = 200 と決定されました。

In [6]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
import numpy as np

''' 
    マイオペのプロダクション・コードから、
    該当部分を抽出して実行しました
    
    ロジスティック回帰モデルを
    ４通りのハイパーパラメータでグリッドサーチ
'''
params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X, y)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum num

{'C': 200}

### (3-2) モデル評価フェーズ

Accuracy = 0.986 と評価されました。

In [7]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score

def __accuracy_score(estimator, X, y):
    y_pred = estimator.predict(X)
    probabilities = estimator.predict_proba(X)
    max_probabilities = np.max(probabilities, axis=1)

    # 予測結果と実際を比較
    bools = y == y_pred
    # しきい値を超えているもののみをTrueにする
    bools = bools == (max_probabilities > 0.5)
    # 正当率を算出
    score = np.sum(bools) / np.size(bools, axis=0)
    return score

''' 
    マイオペのプロダクション・コードから、
    該当部分を抽出して実行しました
    
    反復＝1回、テストデータ比率＝0.25
'''
cv = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.25, random_state=0)
accuracy = np.mean(cross_val_score(estimator, X, y, cv=cv, scoring=__accuracy_score))

In [8]:
accuracy

0.98601238248108236

## (4) ダブルクロスバリデーションを使用する場合

こちらの記述における手順を再現しています。

http://univprof.com/archives/16-06-12-3889388.html

ただし、反復回数は（上記記述から１回ふやして）３回としています。

### (4-0) データを分割

元の訓練データを分割し、各々の反復において使用するためのデータを作成します。

In [9]:
cv = ShuffleSplit(X.shape[0], n_iter=3, test_size=0.333333, random_state=0)
cv

ShuffleSplit(17443, n_iter=3, test_size=0.333333, random_state=0)

In [10]:
'''
    サンプルデータ（CSVから読み込んだ訓練データ全量）を
    ShuffleSplit のパラメータ通り3分割します。
'''
iter_data = []
for train_index, test_index in cv:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    count = X_test.shape[0]
    iter_data.append((X_train, X_test, y_train, y_test, count))

In [11]:
'''
    3分割された分割単位 (iter_data の各要素) ごとに、
    テストデータの件数を表示します。

    合計はサンプル全体の件数になるはずなのですが、
    端数が切り上げられてしまうようなので、
    実際は２件多くなってしまいます。
'''
print('Iteration data count: 1st=%d, 2nd=%d, 3rd=%d, total=%d' % (
    iter_data[0][4], 
    iter_data[1][4], 
    iter_data[2][4],
    iter_data[0][4] + iter_data[1][4] + iter_data[2][4]
))

Iteration data count: 1st=5815, 2nd=5815, 3rd=5815, total=17445


### (4-1) 反復１回目

#### 参考のために、３回の反復を（ループを使わずに）個別に実行させています。ご容赦ください。

In [12]:
''' 
    反復1回目の訓練データ (X_train, y_train) を使用し
    ロジスティック回帰モデルを使用し学習実行
'''
X_train_1, X_test_1, y_train_1, y_test_1, count_1 = iter_data[0]

params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X_train_1, y_train_1)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


{'C': 100}

In [13]:
''' 
    反復1回目の予測結果を取得
'''
y_pred_1 = estimator.predict(X_test_1)
probabilities = estimator.predict_proba(X_test_1)
y_proba_1 = np.max(probabilities, axis=1)

In [14]:
y_pred_1

array([3687, 3402, 3514, ..., 3963, 3813, 3704])

In [15]:
y_proba_1

array([ 0.96999912,  0.95531008,  0.96692472, ...,  0.97500545,
        0.95889908,  0.9768531 ])

In [16]:
''' 
    プロダクション・コードを拝借・・・
'''
def accuracy_score(y, y_pred, y_proba):
    # 予測結果と実際を比較
    bools = y == y_pred
    # しきい値を超えているもののみをTrueにする
    bools = bools == (y_proba > 0.5)
    # 正当率を算出
    score = np.sum(bools) / np.size(bools, axis=0)
    return score

In [17]:
''' 
    参考：反復1回目の単体での正解率
'''
accuracy_1 = accuracy_score(y_test_1, y_pred_1, y_proba_1)
accuracy_1

0.97987962166809972

### (4-2) 反復２回目

In [18]:
''' 
    反復2回目の訓練データ (X_train, y_train) を使用し
    ロジスティック回帰モデルを使用し学習実行
'''
X_train_2, X_test_2, y_train_2, y_test_2, count_2 = iter_data[1]

params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X_train_2, y_train_2)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


{'C': 140}

In [19]:
''' 
    反復2回目の予測結果を取得
'''
y_pred_2 = estimator.predict(X_test_2)
probabilities = estimator.predict_proba(X_test_2)
y_proba_2 = np.max(probabilities, axis=1)

In [20]:
y_pred_2

array([3636, 3433, 3511, ..., 3771, 3544, 3779])

In [21]:
y_proba_2

array([ 0.96614273,  0.98816073,  0.98807663, ...,  0.97451277,
        0.94973569,  0.97417586])

In [22]:
''' 
    参考：反復2回目の単体での正解率
'''
accuracy_2 = accuracy_score(y_test_2, y_pred_2, y_proba_2)
accuracy_2

0.98108340498710234

### (4-3) 反復３回目

In [23]:
''' 
    反復3回目の訓練データ (X_train, y_train) を使用し
    ロジスティック回帰モデルを使用し学習実行
'''
X_train_3, X_test_3, y_train_3, y_test_3, count_3 = iter_data[2]

params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X_train_3, y_train_3)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


{'C': 200}

In [24]:
''' 
    反復3回目の予測結果を取得
'''
y_pred_3 = estimator.predict(X_test_3)
probabilities = estimator.predict_proba(X_test_3)
y_proba_3 = np.max(probabilities, axis=1)

In [25]:
y_pred_3

array([3538, 3713, 3548, ..., 4000, 3547, 3672])

In [26]:
y_proba_3

array([ 0.9744023 ,  0.99402025,  0.98140069, ...,  0.97178126,
        0.98053147,  0.98310901])

In [27]:
''' 
    参考：反復3回目の単体での正解率
'''
accuracy_3 = accuracy_score(y_test_3, y_pred_3, y_proba_3)
accuracy_3

0.98125537403267415

### (4-4) 全ての予測結果に対する正解率を算出

In [28]:
''' 
    反復ごとの予測結果をマージ
    ---> データセット中のすべてのサンプルにおける予測値となります
'''
y_test_all = np.concatenate((y_test_1, y_test_2, y_test_3))
y_pred_all = np.concatenate((y_pred_1, y_pred_2, y_pred_3))
y_proba_all = np.concatenate((y_proba_1, y_proba_2, y_proba_3))

In [29]:
y_pred_all

array([3687, 3402, 3514, ..., 4000, 3547, 3672])

In [30]:
y_proba_all

array([ 0.96999912,  0.95531008,  0.96692472, ...,  0.97178126,
        0.98053147,  0.98310901])

In [31]:
''' 
    全ての予測結果に対する正解率を算出
'''
accuracy_all = accuracy_score(y_test_all, y_pred_all, y_proba_all)
accuracy_all

0.9807394668959587